### Working with MOD13 Vegetation Indices data

I assume this process is going to be similar with 

In [33]:
# Import libraries
import os
import glob
from osgeo import gdal
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import scipy.ndimage
import pandas as pd
import datetime as dt
import rasterio as rio
import re
import geopandas as gpd
import earthpy as et
import earthpy.plot as ep
import earthpy.spatial as es
import earthpy.mask as em
from pyhdf.SD import SD, SDC

In [34]:
# Set input directory, and change working directory
inDir = "C:\\Users\\samgr\\OneDrive\\Desktop\\Masters_Project\\Preparation"   # IMPORTANT: Update to reflect directory on your OS
os.chdir(inDir)                                                               # Change to working directory
outDir = os.path.normpath(os.path.split(inDir)[0] + os.sep + 'output') + '\\' # Create and set output directory
if not os.path.exists(outDir): os.makedirs(outDir)

In [35]:
#haven't got MODIS downloaded yet
MODISFiles = glob.glob('MOD13A3**.hdf') # Search for and create a list of EVI files
MODISFiles

['MOD13A3.A2015001.h19v07.006.2015295114815.hdf',
 'MOD13A3.A2015032.h19v08.006.2015297000353.hdf',
 'MOD13A3.A2015060.h19v08.006.2015298212508.hdf',
 'MOD13A3.A2015091.h19v08.006.2015299041123.hdf',
 'MOD13A3.A2015152.h20v08.006.2015301214710.hdf',
 'MOD13A3.A2015274.h19v07.006.2015317210610.hdf',
 'MOD13A3.A2015305.h19v07.006.2015343135029.hdf',
 'MOD13A3.A2015305.h19v08.006.2015343135001.hdf',
 'MOD13A3.A2015305.h20v08.006.2015343135325.hdf',
 'MOD13A3.A2015335.h19v07.006.2016007181745.hdf',
 'MOD13A3.A2015335.h19v08.006.2016007180603.hdf',
 'MOD13A3.A2015335.h20v08.006.2016007181735.hdf']

In [36]:
print(MODISFiles[0])

MOD13A3.A2015001.h19v07.006.2015295114815.hdf


In [37]:
#Rename MODIS File
productId = MODISFiles[0].split('.')[0]                                         # First: product name
yeardoy = MODISFiles[0].split(productId + '.A')[1].split('.')[0]                #julian date
date = dt.datetime.strptime(yeardoy, '%Y%j').strftime('%d/%m/%Y')               # Convert YYYYDDD to DD/MM/YYYY
tiles_id = MODISFiles[0].split(yeardoy + '.')[1].split('.006')[0]               # Second: layer name
unknown_format = MODISFiles[0].split(tiles_id +'.006.')[1].split('.hdf')[0]     # Third: date


print('Product Name: {}\nLayer ID: {}\nDate of Observation: {}\n(Not sure what this represents: {})'.format(productId, tiles_id, date, unknown_format ))

Product Name: MOD13A3
Layer ID: h19v07
Date of Observation: 01/01/2015
(Not sure what this represents: 2015295114815)


Find out what last part of naming convention represents

In [38]:
# View dataset metadata
with rio.open(MODISFiles[0]) as dataset:
    print(dataset)
    hdf4_meta = dataset.meta

# Notice that there are metadata at the highest level of the file
hdf4_meta

<open DatasetReader name='MOD13A3.A2015001.h19v07.006.2015295114815.hdf' mode='r'>


{'driver': 'HDF4',
 'dtype': 'float_',
 'nodata': None,
 'width': 512,
 'height': 512,
 'count': 0,
 'crs': None,
 'transform': Affine(1.0, 0.0, 0.0,
        0.0, 1.0, 0.0)}

In [39]:
# Print all of the subdatasets in the data
with rio.open(MODISFiles[0]) as dataset:
    crs = dataset.read_crs()
    for name in dataset.subdatasets:
        print(name)

HDF4_EOS:EOS_GRID:MOD13A3.A2015001.h19v07.006.2015295114815.hdf:MOD_Grid_monthly_1km_VI:1 km monthly NDVI
HDF4_EOS:EOS_GRID:MOD13A3.A2015001.h19v07.006.2015295114815.hdf:MOD_Grid_monthly_1km_VI:1 km monthly relative azimuth angle
HDF4_EOS:EOS_GRID:MOD13A3.A2015001.h19v07.006.2015295114815.hdf:MOD_Grid_monthly_1km_VI:1 km monthly pixel reliability
HDF4_EOS:EOS_GRID:MOD13A3.A2015001.h19v07.006.2015295114815.hdf:MOD_Grid_monthly_1km_VI:1 km monthly EVI
HDF4_EOS:EOS_GRID:MOD13A3.A2015001.h19v07.006.2015295114815.hdf:MOD_Grid_monthly_1km_VI:1 km monthly VI Quality
HDF4_EOS:EOS_GRID:MOD13A3.A2015001.h19v07.006.2015295114815.hdf:MOD_Grid_monthly_1km_VI:1 km monthly red reflectance
HDF4_EOS:EOS_GRID:MOD13A3.A2015001.h19v07.006.2015295114815.hdf:MOD_Grid_monthly_1km_VI:1 km monthly NIR reflectance
HDF4_EOS:EOS_GRID:MOD13A3.A2015001.h19v07.006.2015295114815.hdf:MOD_Grid_monthly_1km_VI:1 km monthly blue reflectance
HDF4_EOS:EOS_GRID:MOD13A3.A2015001.h19v07.006.2015295114815.hdf:MOD_Grid_monthly_1

In [40]:
file_name = 'MOD13A3.A2015001.h19v07.006.2015295114815.hdf'
file = SD(file_name, SDC.READ)

print( file.info() )

# MYD06_L2.A2007219.2010.006.2014053202546.hdf

(11, 6)


In [41]:
datasets_dic = file.datasets()

for idx,sds in enumerate(datasets_dic.keys()):
    print(idx,sds)

0 1 km monthly NDVI
1 1 km monthly EVI
2 1 km monthly VI Quality
3 1 km monthly red reflectance
4 1 km monthly NIR reflectance
5 1 km monthly blue reflectance
6 1 km monthly MIR reflectance
7 1 km monthly view zenith angle
8 1 km monthly sun zenith angle
9 1 km monthly relative azimuth angle
10 1 km monthly pixel reliability


In [42]:
sds_obj = file.select(0) # select sds

data = sds_obj.get() # get sds data
print(data)

[[ 942  909  877 ...  938  891  931]
 [ 926  922  890 ...  850  943  931]
 [ 907  909  896 ...  963  947  955]
 ...
 [2996 3215 3117 ... 4299 4433 4270]
 [2787 3089 3035 ... 3453 3805 3320]
 [3031 2990 3132 ... 4372 4316 3319]]


In [43]:
import pprint

pprint.pprint( sds_obj.attributes() )

{'_FillValue': -3000,
 'add_offset': 0.0,
 'add_offset_err': 0.0,
 'calibrated_nt': 5,
 'long_name': '1 km monthly NDVI',
 'scale_factor': 10000.0,
 'scale_factor_err': 0.0,
 'units': 'NDVI',
 'valid_range': [-2000, 10000]}


In [44]:
for key, value in sds_obj.attributes().items():
    print(key, value)
    if key == 'add_offset':
        add_offset = value  
    if key == 'scale_factor':
        scale_factor = value

long_name 1 km monthly NDVI
units NDVI
valid_range [-2000, 10000]
_FillValue -3000
scale_factor 10000.0
scale_factor_err 0.0
add_offset 0.0
add_offset_err 0.0
calibrated_nt 5


In [45]:
data = (data - add_offset) * scale_factor
print(data)

[[ 9420000.  9090000.  8770000. ...  9380000.  8910000.  9310000.]
 [ 9260000.  9220000.  8900000. ...  8500000.  9430000.  9310000.]
 [ 9070000.  9090000.  8960000. ...  9630000.  9470000.  9550000.]
 ...
 [29960000. 32150000. 31170000. ... 42990000. 44330000. 42700000.]
 [27870000. 30890000. 30350000. ... 34530000. 38050000. 33200000.]
 [30310000. 29900000. 31320000. ... 43720000. 43160000. 33190000.]]


In [46]:
print(data.shape)

(1200, 1200)
